`Monta Drive`

In [ ]:
# @title

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


`Estrazione dataset`





In [ ]:
# @title


!scp '/content/gdrive/My Drive/FishersDetector/mesa2025_fisher_detection.zip' '/content/mesa2025_fisher_detection.zip'

!unzip '/content/mesa2025_fisher_detection.zip' -d '/content/'

Output streaming troncato alle ultime 5000 righe.
  inflating: /content/detection_mix/images/train/9999950_00000_d_0000002_jpg.rf.88dccb2f6317a8f1a77ed28df1c794df.jpg  
  inflating: /content/detection_mix/images/train/9999940_00000_d_0000005_jpg.rf.26427f3f6b424a2aefd483a24640e7f5.jpg  
  inflating: /content/detection_mix/images/train/9999953_00000_d_0000019_jpg.rf.374cd4f8f71c33686565de57310b9fb7.jpg  
  inflating: /content/detection_mix/images/train/0000256_04356_d_0000036_jpg.rf.fb6e4b191e5d30d41c850391fb98bfca.jpg  
  inflating: /content/detection_mix/images/train/0000293_01001_d_0000927_jpg.rf.22085ebe93cb0bc743c9f4b0bff9bb71.jpg  
  inflating: /content/detection_mix/images/train/9999966_00000_d_0000010_jpg.rf.75568174c7908ef820707da5d1072fa1.jpg  
  inflating: /content/detection_mix/images/train/0000195_01418_d_0000133_jpg.rf.7c035fa11d98820cb69251955eddb98c.jpg  
  inflating: /content/detection_mix/images/train/9999974_00000_d_0000055_jpg.rf.81725e4af32c9851c8b6e191cfbdcdb6.jpg 

`Install packages`

In [ ]:
# @title


!pip install ultralytics
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00


`Train manuale con diverse configurazioni`

In [ ]:
# @title

from ultralytics import YOLO
import torch
import sys

# controllo gpu
if (torch.cuda.is_available() == False):
    print("GPU non disponibile")
    sys.exit()
else:
    print("GPU disponibile")
    print("Nome GPU:", torch.cuda.get_device_name(0))

# carico il modello
model = YOLO("yolo11l.pt")

# dataset
config_path = '/content/detection_mix/detect.yaml'

# configurazioni da confrontare
configurazioni = [

    # 1. configurazione di base
    {
        "name": "config1_base_auto",
        "project": "/content/runs/train",
        "run_name": "config1_base_auto",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.01, # learning rate iniziale
        "momentum": 0.937, # momentum standard
        "optimizer": "auto",
        "dfl": 1.5, # DFL standard

        "patience": 25,
        "save_period": 25,
    },

    # 2. configurazione di base con SGD
    {
        "name": "config2_base_sgd",
        "project": "/content/runs/train",
        "run_name": "config2_base_sgd",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.01, # learning rate iniziale
        "momentum": 0.937, # momentum standard
        "optimizer": "SGD",
        "dfl": 1.5, # DFL standard

        "patience": 25,
        "save_period": 25,

    },

    # 3. configurazione con learning rate basso e SGD
    {
        "name": "config3_lrbasso",
        "project": "/content/runs/train",
        "run_name": "config3_lrbasso",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.001,
        "momentum": 0.937,
        "optimizer": "SGD",

        "patience": 25,
        "save_period": 25,

    },

    # 4. configurazione con learning rate alto e SGD
    {
        "name": "config4_lralto",
        "project": "/content/runs/train",
        "run_name": "config4_lralto",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.1,
        "momentum": 0.937,
        "optimizer": "SGD",

        "patience": 25,
        "save_period": 25,

    },

    # 5. configurazione con momentum SGD
    {
        "name": "config5_momentum",
        "project": "/content/runs/train",
        "run_name": "config5_momentum",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.01,
        "momentum": 0.9, # momentum ridotto
        "optimizer": "SGD",

        "patience": 25,
        "save_period": 25,

    },

    # 6. configurazione con DFL aumentato
    {
        "name": "config6_dflalto",
        "project": "/content/runs/train",
        "run_name": "config6_dflalto",

        "imgsz": 640,
        "batch": 16,
        "epochs": 50,

        "lr0": 0.01,
        "momentum": 0.937,
        "optimizer": "SGD",
        "dfl": 2.0, # DFL aumentato

        "patience": 25,
        "save_period": 25,

    },

    # 7. configurazione con imgsz ridotto + batch aumentato
    {
        "name": "config7_imgszridotto",
        "project": "/content/runs/train",
        "run_name": "config7_imgszridotto",

        "imgsz": 512, # ridotto
        "batch": 32, # aumentato
        "epochs": 50,

        "lr0": 0.01,
        "momentum": 0.937,
        "optimizer": "SGD",
        "dfl": 1.5,

        "patience": 25,
        "save_period": 25,

        }
]

# stampa menu configurazioni
print("Configurazioni disponibili:\n")
for i, cfg in enumerate(configurazioni):
    print(f"[{i}] {cfg['name']}")

# inizializza con valore non valido
config_to_run = -1

# loop finché non inserisci un indice valido
while not (0 <= config_to_run < len(configurazioni)):
    try:
        config_to_run = int(input("\nSeleziona il numero della configurazione da eseguire: "))
        if not (0 <= config_to_run < len(configurazioni)):
            print("Numero non valido, riprova.")
    except ValueError:
        print("Inserisci un numero valido.")
        config_to_run = -1  # rimane invalido

# esegui configurazione scelta
config = configurazioni[config_to_run]
print(f"\n▶ Avvio addestramento con {config['name']}...\n")

model.train(
    device=0,
    data=config_path,
    project=config["project"],
    name=config["run_name"],
    imgsz=config["imgsz"],
    batch=config["batch"],
    epochs=config["epochs"],
    lr0=config["lr0"],
    momentum=config["momentum"],
    optimizer=config["optimizer"],
    dfl=config.get("dfl", 1.5),
    patience=config["patience"],
    save_period=config["save_period"],
)


print(f"\n Addestramento con {config['name']} completato")


GPU disponibile
Nome GPU: NVIDIA A100-SXM4-40GB
Configurazioni disponibili:

[0] config1_base_auto
[1] config2_base_sgd
[2] config3_lrbasso
[3] config4_lralto
[4] config5_momentum
[5] config6_dflalto
[6] config7_imgszridotto


`Download di tutta la cartella train (tutti i risultati)`

In [ ]:
# @title

import shutil
import os
from google.colab import files

# Percorso cartella
folder_path = '/content/runs/train'

# Verifica che la cartella esista
if os.path.exists(folder_path):
    print(f"Cartella trovata: {folder_path}")

    # Crea un archivio zip di tutta la cartella train
    zip_name = '/content/yolo_training_results_congif_best'
    shutil.make_archive(zip_name, 'zip', folder_path)

    # Download del file zip
    files.download(f'{zip_name}.zip')
else:
    print(f"Cartella non trovata: {folder_path}")




Cartella trovata: /content/runs/train


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

`Ricerca parametri ottimali per detection, automatica con tune`

In [ ]:
# @title


from ultralytics import YOLO
import torch
import sys

# controllo gpu
if not torch.cuda.is_available():
    print("GPU non disponibile")
    sys.exit()
else:
    print("GPU disponibile")
    print("Nome GPU:", torch.cuda.get_device_name(0))

# carico il modello
model = YOLO("yolo11l.pt")

# dataset path
config_path = '/content/detection_mix/detect.yaml'



# avvia ottimizzazione automatica
results = model.tune(
    device=0,
    data=config_path,

    # parametri del tuning
    epochs=30,           # epoche per ogni test
    iterations=25,       # numero di combinazioni da testare

    # ottimizza automaticamente questi parametri:
    # - lr0 (learning rate iniziale)
    # - lrf (learning rate finale)
    # - momentum (per SGD)
    # - weight_decay (regolarizzazione)
    # - warmup_epochs, warmup_momentum
    # - box (peso loss bounding box) - IMPORTANTE per detection
    # - cls (peso loss classificazione)
    # - dfl (Distribution Focal Loss) - tuo parametro testato
    # - augmentations (hsv, translate, scale, flip, mosaic)


    # parametri fissi
    imgsz=512,          # image size
    batch=48,           # batch size

    # salvataggio
    project="/content/runs/tune_detection",
    name="detection_auto_tune",

    # opzioni
    plots=True,         # genera grafici
    save=True,          # salva il modello migliore
    patience=15,        # early stopping
    cache=True,         # cache dataset per velocità

    # output
    verbose=True        # mostra dettagli
)



# download dei risultati
import os
import zipfile
from google.colab import files


# crea zip con tutti i risultati
zip_path = "/content/yolo_detection_tune_results.zip"
results_folder = "/content/runs/tune_detection/detection_auto_tune/"

if os.path.exists(results_folder):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_list in os.walk(results_folder):
            for file in files_list:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, "/content/runs/tune_detection/")
                zipf.write(file_path, arcname)

    print(f"Zip DETECTION creato: {zip_path}")

    # avvia download
    files.download(zip_path)





Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU disponibile
Nome GPU: NVIDIA A100-SXM4-40GB
🎯 Avvio YOLO Tune - Ottimizzazione automatica iperparametri per Detection
Tuner: Initialized Tuner instance with 'tune_dir=/content/runs/tune_detection/detection_auto_tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/25 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosai

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download completato!
🎯 Usa il modello best.pt per detection ottimizzata!
🔧 I parametri nel file .yaml sono ottimizzati per il tuo dataset detection


`Ricerca parametri ottimali per classification, automatica con tune`

In [ ]:
# @title

from ultralytics import YOLO
import torch
import sys

# controllo gpu
if not torch.cuda.is_available():
    print("GPU non disponibile")
    sys.exit()
else:
    print("GPU disponibile")
    print("Nome GPU:", torch.cuda.get_device_name(0))

# carico il modello
model = YOLO("yolo11l-cls.pt")

# dataset path
dataset_path = '/content/classification_mix'

results = model.tune(
    device=0,
    data=dataset_path,
    epochs=30,           # epoche per ogni test
    iterations=30,       # numero di combinazioni da testare
    imgsz=512,
    batch=32,
    project="/content/runs/tune",
    name="auto_tune",
    plots=True,         # genera grafici
    save=True,          # salva il modello migliore
    patience=10,        # early stopping
    cache=True,         # cache dataset per velocità
)

# ottimizza questi parametri automaticamente:
    # - lr0 (learning rate iniziale)
    # - momentum
    # - weight_decay
    # - warmup_epochs
    # - warmup_momentum
    # - cls (per classification)


    # output
    #verbose=True        # mostra dettagli


# download dei risultati
import os
import zipfile
from google.colab import files



# crea zip con tutti i risultati
zip_path = "/content/yolo_tune_results.zip"
results_folder = "/content/runs/tune/"

if os.path.exists(results_folder):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_list in os.walk(results_folder):
            for file in files_list:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, "/content/runs/")
                zipf.write(file_path, arcname)

    print(f"Zip creato: {zip_path}")


    # avvia download
    files.download(zip_path)
    print(f"Download completato")

else:
    print(f"Cartella risultati non trovata: {results_folder}")


GPU disponibile
Nome GPU: NVIDIA A100-SXM4-40GB
🎯 Avvio YOLO Tune - Ottimizzazione automatica iperparametri
Tuner: Initialized Tuner instance with 'tune_dir=/content/runs/tune/auto_tune2'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/30 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}
Saved /content/runs/tune/auto_tune2/tune_scatter_plots.png
Saved /content/runs/tune/auto_tune2/tune_fitness.png

Tuner: 1/30 iterations complete ✅ (86.83s)
Tuner: Results saved to /content/runs/tune/auto_tune2
Tuner: Best fitness=1.0 observed at iteration 1
Tuner: Be

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download completato!
🎯 Usa il modello best.pt per i tuoi progetti!


`Train detection con parametri migliori trovati`

In [ ]:
# @title

from ultralytics import YOLO
import torch
import sys

# controllo gpu
if (torch.cuda.is_available() == False):
    print("GPU non disponibile")
    sys.exit()
else:
    print("GPU disponibile")
    print("Nome GPU:", torch.cuda.get_device_name(0))

# carico modello
model = YOLO("yolo11l.pt")

# dataset
config_path = '/content/detection_mix/detect.yaml'

# configurazione con parametri migliori
config = {
        "name": "config_best",
        "project": "/content/runs/train",
        "run_name": "config_best",

        "imgsz": 640,
        "batch": 16,
        "epochs": 100,
        "lr0": 0.001, # learning ridotto
        "momentum": 0.937, # momentum ridotto
        "optimizer": "auto",
        "dfl": 1.5, # DFL standard

        "patience": 25,
        "save_period": 33
}

model.train(
    device=0,
    data=config_path,
    project=config["project"],
    name=config["run_name"],
    imgsz=config["imgsz"],
    batch=config["batch"],
    epochs=config["epochs"],
    lr0=config["lr0"],
    momentum=config["momentum"],
    optimizer=config["optimizer"],
    dfl=config.get("dfl", 1.5),
    patience=config["patience"],
    save_period=config["save_period"],
)

print("Addestramento completato")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU disponibile
Nome GPU: Tesla T4
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/detection_mix/detect.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, 